## Library Imports

In [1]:
from time import time
notebook_start_time = time()

In [2]:
import os
import re
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import nn, optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader as DL
from torch.nn.utils import weight_norm as WN
from torchvision import models, transforms

from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

## Constants and Utilities

In [3]:
SEED = 0
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
PATH = "../input/petfinder-pawpularity-score"
PRETRAINED_WEIGHTS_PATH = "../input/pretrained-model-weights"

NUM_FOLDS = 10
IMAGE_SIZE = 224
TRANSFORM = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize([0.485, 0.456, 0.406],
                                                     [0.229, 0.224, 0.225]),
                                ])
sc_y = StandardScaler()

model_names = ["densenet", "resnet", "mobilenet", "vgg"]
trained_annl_weights_paths = ["../input/petfinder-densenet-train-10fcv",
                              "../input/petfinder-resnet-train-10fcv",
                              "../input/petfinder-mobilenet-train-10fcv",
                              "../input/petfinder-vgg-train-10fcv"]
feature_lengths = [1664, 2048, 1280, 2048]

In [4]:
def breaker(num=50, char="*") -> None:
    print("\n" + num*char + "\n")


def head(x, no_of_ele=5) -> None:
    print(x[:no_of_ele])


def get_filenames_and_targets(path: str) -> tuple:
    df = pd.read_csv(os.path.join(path, "train.csv"), engine="python")
    filenames = df.iloc[:, 0].copy().values
    targets  = df.iloc[:, -1].copy().values
    return filenames, targets


def get_filenames(path: str) -> np.ndarray:
    df = pd.read_csv(os.path.join(path, "test.csv"), engine="python")
    filenames  = df["Id"].copy().values
    return filenames


def get_image(path: str, name: str, size: int) -> np.ndarray:
    image = cv2.imread(os.path.join(path, name + ".jpg"), cv2.IMREAD_COLOR)
    image = cv2.cvtColor(src=image, code=cv2.COLOR_BGR2RGB)
    image = cv2.resize(src=image, dsize=(size, size), interpolation=cv2.INTER_AREA)
    return image


def make_submission(path: str, y_pred: np.ndarray) -> None:
    submission = pd.read_csv(os.path.join(path, "sample_submission.csv"), engine="python")
    submission["Pawpularity"] = y_pred
    submission.to_csv("./submission.csv", index=False)

## Dataset Templates

In [5]:
class ImageDS(Dataset):
    def __init__(self, base_path=None, filenames=None, image_size=None, transform=None):
        self.base_path = base_path
        self.filenames = filenames
        self.image_size = image_size
        self.transform = transform
    
    def __len__(self):
        return self.filenames.shape[0]
    
    def __getitem__(self, idx):
        image = get_image(self.base_path, self.filenames[idx], self.image_size)
        return self.transform(image)

    
class FeatureDS(Dataset):
    def __init__(self, features=None):
        self.features = features
    
    def __len__(self):
        return self.features.shape[0]
    
    def __getitem__(self, idx):
        return torch.FloatTensor(self.features[idx])

## Build Models

In [6]:
def build_models(model_name: str, IL: int, pretrained: bool, seed: int):

    class ImageModel(nn.Module):
        def __init__(self, model_name=None, pretrained=False):
            super(ImageModel, self).__init__()

            if re.match(r"^resnet$", model_name, re.IGNORECASE):
                self.features = models.resnet50(pretrained=pretrained, progress=True)
                if pretrained:
                    self.freeze()
                self.features = nn.Sequential(*[*self.features.children()][:-1])
                self.features.add_module("Flatten", nn.Flatten())

                self.in_features = self.features[-3][2].bn3.num_features


            elif re.match(r"^vgg$", model_name, re.IGNORECASE):
                self.features = models.vgg16_bn(pretrained=pretrained, progress=True)
                if pretrained:
                    self.freeze()
                self.features = nn.Sequential(*[*self.features.children()][:-2])
                self.features.add_module("Adaptive Average Pool", nn.AdaptiveAvgPool2d(output_size=(2, 2)))
                self.features.add_module("Flatten", nn.Flatten())

                self.in_features = self.features[-3][41].num_features * 2 * 2


            elif re.match(r"^mobilenet$", model_name, re.IGNORECASE):
                self.features = models.mobilenet_v2(pretrained=pretrained, progress=True)
                if pretrained:
                    self.freeze()
                self.features = nn.Sequential(*[*self.features.children()][:-1])
                self.features.add_module("Adaptive Average Pool", nn.AdaptiveAvgPool2d(output_size=(1, 1)))
                self.features.add_module("Flatten", nn.Flatten())

                self.in_features = self.features[-3][-1][1].num_features


            elif re.match(r"^densenet$", model_name, re.IGNORECASE):
                self.features = models.densenet169(pretrained=pretrained, progress=True)
                if pretrained:
                    self.freeze()
                self.features = nn.Sequential(*[*self.features.children()][:-1])
                self.features.add_module("Adaptive Average Pool", nn.AdaptiveAvgPool2d(output_size=(1, 1)))
                self.features.add_module("Flatten", nn.Flatten())

                self.in_features = self.features[0].norm5.num_features

        def freeze(self):
            for params in self.parameters():
                    params.requires_grad = False

        def forward(self, x):
            return self.features(x)
    
    
    breaker()
    print("Building Vision Model ...")
    print("\n{} Features".format(model_name))
    
    torch.manual_seed(seed)
    vision_model = ImageModel(model_name=model_name, pretrained=pretrained)
    
    
    class ANN(nn.Module):
        def __init__(self, IL=None):
            super(ANN, self).__init__()

            self.predictor = nn.Sequential()
            self.predictor.add_module("BN", nn.BatchNorm1d(num_features=IL, eps=1e-5))
            self.predictor.add_module("FC", WN(nn.Linear(in_features=IL, out_features=1)))

        def get_optimizer(self, lr=1e-3, wd=0):
            params = [p for p in self.parameters() if p.requires_grad]
            return optim.Adam(params, lr=lr, weight_decay=wd)

        def get_plateau_scheduler(self, optimizer=None, patience=5, eps=1e-8):
            return optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, patience=patience, eps=eps, verbose=True)

        def forward(self, x):
            return self.predictor(x)
    
    breaker()
    print("Building Model ...")
    print("\n{} -> 1".format(IL))
    
    torch.manual_seed(seed)
    ann_model = ANN(IL=IL)
    
    return vision_model, ann_model

## Feature Getter and Predict Helper

In [7]:
def get_features(model=None, dataloader=None, feature_length=None, path=None):
    model.load_state_dict(torch.load(path, map_location=DEVICE))
    model.to(DEVICE)
    model.eval()

    features = torch.zeros(1, feature_length).to(DEVICE)
    for X in dataloader:
        X = X.to(DEVICE)
        with torch.no_grad():
            output = model(X)
        features = torch.cat((features, output.view(-1, feature_length)), dim=0)
    
    return features[1:].detach().cpu().numpy()


def predict_batch(model=None, dataloader=None, mode="test", path=None):
    model.load_state_dict(torch.load(path, map_location=DEVICE)["model_state_dict"])
    model.to(DEVICE)
    model.eval()

    y_pred = torch.zeros(1, 1).to(DEVICE)
    if re.match(r"valid", mode, re.IGNORECASE):
        for X, _ in dataloader:
            X = X.to(DEVICE)
            with torch.no_grad():
                output = model(X)
            y_pred = torch.cat((y_pred, output.view(-1, 1)), dim=0)
    elif re.match(r"test", mode, re.IGNORECASE):
        for X in dataloader:
            X = X.to(DEVICE)
            with torch.no_grad():
                output = model(X)
            y_pred = torch.cat((y_pred, output.view(-1, 1)), dim=0)
    
    return y_pred[1:].detach().cpu().numpy()

## Generate Submission

In [8]:
def submit():
    breaker()
    print("Reading Data ...")
    
    ts_filenames = get_filenames(PATH)
    filenames, targets = get_filenames_and_targets(PATH)
    
    
    breaker()
    print("Obtaining Test Image Features ...")
    
    ts_image_data_setup = ImageDS(base_path=os.path.join(PATH, "test"), 
                                  filenames=ts_filenames, 
                                  image_size=IMAGE_SIZE, 
                                  transform=TRANSFORM)
    ts_image_data = DL(ts_image_data_setup, batch_size=64, shuffle=False)
    
    final_y_pred = np.zeros((len(ts_filenames), 1))
    for i in range(len(model_names)):
        VisionModel, ANNModel = build_models(model_name=model_names[i], IL=feature_lengths[i], pretrained=False, seed=SEED)

        ts_features = get_features(model=VisionModel, dataloader=ts_image_data, 
                                   feature_length=feature_lengths[i], 
                                   path=os.path.join(PRETRAINED_WEIGHTS_PATH, "{}_state.pt".format(model_names[i])))
    
    
        breaker()
        print("Making Predictions on Test Features ...")

        fold = 1
        for tr_idx, va_idx in KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=SEED).split(filenames):
            fold += 1
            if fold == 3:
                break

        tr_targets = targets[tr_idx]
        tr_targets = tr_targets.reshape(-1, 1)
        tr_targets = sc_y.fit_transform(tr_targets)

        ts_feature_data_setup = FeatureDS(features=ts_features)
        ts_feature_data = DL(ts_feature_data_setup, batch_size=512, shuffle=False)

        y_pred = predict_batch(model=ANNModel, dataloader=ts_feature_data, mode="test",
                               path=os.path.join(trained_annl_weights_paths[i], "Fold_{}_state.pt".format(fold)))
        y_pred = sc_y.inverse_transform(y_pred)

        final_y_pred += y_pred

    final_y_pred = final_y_pred / len(model_names)
    
    
    breaker()
    print("Generating Submission File ...")
    make_submission(PATH, final_y_pred)
    breaker()

submit()


**************************************************

Reading Data ...

**************************************************

Obtaining Test Image Features ...

**************************************************

Building Vision Model ...

densenet Features

**************************************************

Building Model ...

1664 -> 1

**************************************************

Making Predictions on Test Features ...

**************************************************

Building Vision Model ...

resnet Features

**************************************************

Building Model ...

2048 -> 1

**************************************************

Making Predictions on Test Features ...

**************************************************

Building Vision Model ...

mobilenet Features

**************************************************

Building Model ...

1280 -> 1

**************************************************

Making Predictions on Test Features ...

*********************

In [9]:
breaker()
print("Notebook Run Time : {:.2f} minutes".format((time()-notebook_start_time)/60))
breaker()


**************************************************

Notebook Run Time : 0.32 minutes

**************************************************

